<a href="https://colab.research.google.com/github/lukkychan/scaling_to_high/blob/main/Welcome_to_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!mkdir original_video
!mkdir org_no_sound
!mkdir -p frames/1080p
!mkdir -p frames/360p
!mkdir scaled
!mkdir predicted
!mkdir Pre_orginal
!mkdir dataset
!mkdir -p dataset/1080p
!mkdir -p dataset/360s

https://www.youtube.com/watch?v=eDTEiIC0_rE&pp=ygUSc3B5IHRyYWlsZXIgdGVsdWd1

Downloading the videos

In [1]:
!wget -O "1080p.mp4" "https://cdn6.savetube.me/media/eDTEiIC0_rE/spy-trailer-telugu-nikhil-siddharth-iswarya-menon-garry-bh-charantej-uppalapati-1080-ytshorts.savetube.me.mp4"

--2023-06-30 07:09:44--  https://cdn6.savetube.me/media/eDTEiIC0_rE/spy-trailer-telugu-nikhil-siddharth-iswarya-menon-garry-bh-charantej-uppalapati-1080-ytshorts.savetube.me.mp4
Resolving cdn6.savetube.me (cdn6.savetube.me)... 139.99.45.74
Connecting to cdn6.savetube.me (cdn6.savetube.me)|139.99.45.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19116714 (18M) [video/mp4]
Saving to: ‘1080p.mp4’

1080p.mp4           100%[===================>]  18.23M  7.41MB/s    in 2.5s    

2023-06-30 07:09:49 (7.41 MB/s) - ‘1080p.mp4’ saved [19116714/19116714]



In [2]:
!wget -O "360p.mp4" "https://rr1---sn-npoe7nz7.googlevideo.com/videoplayback?expire=1688130533&ei=hX-eZLbxE87yrQH-15eADw&ip=139.99.45.74&id=o-ABcVHdyuIRW-l7JWY6kvjKugTdbRE9-T7o4v0aBxASIO&itag=18&source=youtube&requiressl=yes&mh=5x&mm=31%2C29&mn=sn-npoe7nz7%2Csn-npoeene6&ms=au%2Crdu&mv=m&mvi=1&pl=17&initcwndbps=117500&spc=Ul2Sq_HzubF7d7fy3t7PLZOutGLOXNY&vprv=1&svpuc=1&mime=video%2Fmp4&gir=yes&clen=7096879&ratebypass=yes&dur=124.435&lmt=1687436485346821&mt=1688108497&fvip=1&fexp=24007246&beids=24350017&c=ANDROID&txp=6319224&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cspc%2Cvprv%2Csvpuc%2Cmime%2Cgir%2Cclen%2Cratebypass%2Cdur%2Clmt&sig=AOq0QJ8wRQIgL-4zeBPHW9PqAc_zrhsbhQYsUkKbFMVnNH7HOtpGOewCIQCICZ2yXFVmmyELjOS48SciiC2P5Dw-_gC_jDypd07tZw%3D%3D&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cinitcwndbps&lsig=AG3C_xAwRgIhAJAwWzU5sQ7hvvLN8AnTkqgVzNqVwzwPSUL76HrwOaU2AiEAp6yQ_l6dbTg3RErd7xaJ8uWYpEaUoyoLaRuT4pvt2Fs%3D&title=SPY%20Trailer%20(Telugu)%20|%20Nikhil%20Siddharth%20|%20%20Iswarya%20Menon%20|%20Garry%20BH%20|%20Charantej%20Uppalapati"

--2023-06-30 07:09:52--  https://rr1---sn-npoe7nz7.googlevideo.com/videoplayback?expire=1688130533&ei=hX-eZLbxE87yrQH-15eADw&ip=139.99.45.74&id=o-ABcVHdyuIRW-l7JWY6kvjKugTdbRE9-T7o4v0aBxASIO&itag=18&source=youtube&requiressl=yes&mh=5x&mm=31%2C29&mn=sn-npoe7nz7%2Csn-npoeene6&ms=au%2Crdu&mv=m&mvi=1&pl=17&initcwndbps=117500&spc=Ul2Sq_HzubF7d7fy3t7PLZOutGLOXNY&vprv=1&svpuc=1&mime=video%2Fmp4&gir=yes&clen=7096879&ratebypass=yes&dur=124.435&lmt=1687436485346821&mt=1688108497&fvip=1&fexp=24007246&beids=24350017&c=ANDROID&txp=6319224&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cspc%2Cvprv%2Csvpuc%2Cmime%2Cgir%2Cclen%2Cratebypass%2Cdur%2Clmt&sig=AOq0QJ8wRQIgL-4zeBPHW9PqAc_zrhsbhQYsUkKbFMVnNH7HOtpGOewCIQCICZ2yXFVmmyELjOS48SciiC2P5Dw-_gC_jDypd07tZw%3D%3D&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cinitcwndbps&lsig=AG3C_xAwRgIhAJAwWzU5sQ7hvvLN8AnTkqgVzNqVwzwPSUL76HrwOaU2AiEAp6yQ_l6dbTg3RErd7xaJ8uWYpEaUoyoLaRuT4pvt2Fs%3D&title=SPY%20Trailer%20(Telugu)%20%7C%20Nikhil%20Siddharth%20%

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!mkdir '/content/drive/MyDrive/model'

In [16]:
file_path = '/content/org_no_sound/360p.mp4'  # Specify the path to your file
drive_path = '/content/drive/MyDrive/model/360p.mp4'  # Specify the destination path in Google Drive

# Copy the file to Google Drive
!cp $file_path $drive_path

In [12]:
!mv "1080p.mp4" "/content/original_video/"
!mv "360p.mp4" "/content/original_video/"

Removing the audios

In [ ]:
!pip install moviepy

In [13]:
from moviepy.editor import VideoFileClip

def remove_audio(input_video_path, output_video_path):
    # Load the video clip
    video = VideoFileClip(input_video_path)

    # Remove the audio
    video_without_audio = video.without_audio()

    # Save the video without audio
    video_without_audio.write_videofile(output_video_path, codec='libx264')

    # Close the video clip
    video.close()

# Specify the paths to the input video and desired output video
input_video_path = '/content/original_video/1080p.mp4'
output_video_path = '/content/org_no_sound/1080p.mp4'

# Call the function to remove audio
remove_audio(input_video_path, output_video_path)
print("done.")


Moviepy - Building video /content/org_no_sound/1080p.mp4.
Moviepy - Writing video /content/org_no_sound/1080p.mp4



t: 100%|█████████▉| 2984/2987 [06:53<00:00, 28.04it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/original_video/1080p.mp4, 6220800 bytes wanted but 0 bytes read,at frame 2985/2987, at time 124.38/124.44 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready /content/org_no_sound/1080p.mp4
done.


In [14]:
from moviepy.editor import VideoFileClip

def remove_audio(input_video_path, output_video_path):
    # Load the video clip
    video = VideoFileClip(input_video_path)

    # Remove the audio
    video_without_audio = video.without_audio()

    # Save the video without audio
    video_without_audio.write_videofile(output_video_path, codec='libx264')

    # Close the video clip
    video.close()

# Specify the paths to the input video and desired output video
input_video_path = '/content/original_video/360p.mp4'
output_video_path = '/content/org_no_sound/360p.mp4'

# Call the function to remove audio
remove_audio(input_video_path, output_video_path)
print("done.")


Moviepy - Building video /content/org_no_sound/360p.mp4.
Moviepy - Writing video /content/org_no_sound/360p.mp4



t: 100%|█████████▉| 2978/2987 [00:46<00:00, 205.44it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/original_video/360p.mp4, 691200 bytes wanted but 0 bytes read,at frame 2985/2987, at time 124.38/124.44 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready /content/org_no_sound/360p.mp4
done.


Creating Frames

In [7]:
import cv2

# Specify the path to the video file
video_path = "/content/drive/MyDrive/model/360p.mp4"

# Specify the output folder to save the frames
output_folder = "/content/frames/360p/"

# Specify the maximum number of frames to extract
max_frames = 150

# Open the video file
video = cv2.VideoCapture(video_path)

# Get the total number of frames in the video
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
print(total_frames)

# Calculate the frame interval to evenly sample the frames
frame_interval = max(total_frames // max_frames, 1)

# Initialize a counter to keep track of the extracted frames
frame_count = 0

# Loop through the frames and extract the desired number of frames
while frame_count < total_frames:
    # Read the current frame
    ret, frame = video.read()

    # Check if the frame was successfully read
    if not ret:
        break

    # Save the frame as an image file
    frame_path = f"{output_folder}{frame_count:03d}.jpg"
    cv2.imwrite(frame_path, frame)

    # Increment the frame count
    frame_count += 1

    # Move to the next frame based on the frame interval
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_count * frame_interval)

# Release the video capture object
video.release()
print("done.")

2987
done.


In [8]:
import cv2

# Specify the path to the video file
video_path = "/content/drive/MyDrive/model/1080p.mp4"

# Specify the output folder to save the frames
output_folder = "/content/frames/1080p/"

# Specify the maximum number of frames to extract
max_frames = 150

# Open the video file
video = cv2.VideoCapture(video_path)

# Get the total number of frames in the video
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Calculate the frame interval to evenly sample the frames
frame_interval = max(total_frames // max_frames, 1)

# Initialize a counter to keep track of the extracted frames
frame_count = 0

# Loop through the frames and extract the desired number of frames
while frame_count < total_frames:
    # Read the current frame
    ret, frame = video.read()

    # Check if the frame was successfully read
    if not ret:
        break

    # Save the frame as an image file
    frame_path = f"{output_folder}{frame_count:03d}.jpg"
    cv2.imwrite(frame_path, frame)


    # Increment the frame count
    frame_count += 1

    # Move to the next frame based on the frame interval
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_count * frame_interval)

# Release the video capture object
video.release()
print("done.")

done.


Scaling the images to 1080p

In [9]:
import os
import subprocess

def scale_images(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of all image files in the input folder
    image_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

    for image_file in image_files:
        # Construct the input and output file paths
        input_path = os.path.join(input_folder, image_file)
        output_path = os.path.join(output_folder, image_file)

        # Execute FFmpeg command to scale the image to 1920x1080
        command = ['ffmpeg', '-i', input_path, '-vf', 'scale=1920:1080', output_path]
        subprocess.call(command)

# Specify the paths to the input folder and output folder
input_folder = '/content/frames/360p/'
output_folder = '/content/scaled/'

# Call the function to scale the images
scale_images(input_folder, output_folder)
print("scaling done.")


scaling done.


In [10]:
import gc

# Your program code here

# Perform memory cleanup
gc.collect()


21

Final splitted dataset

In [11]:
#1080p
import os
import cv2

# Specify the directory containing the original 1080p images
original_dir = '/content/frames/1080p/'

# Create a directory to save the cropped images
#os.makedirs('path/to/save/cropped/images', exist_ok=True)
counter = 1
# Iterate over the images in the original directory
for filename in os.listdir(original_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions if needed
        # Load the original 1080p image
        original_path = os.path.join(original_dir, filename)
        original_image = cv2.imread(original_path)

        # Calculate the dimensions of each cropped part
        image_height, image_width, _ = original_image.shape
        crop_height = image_height // 3
        crop_width = image_width // 3

        # Crop the image into 9 equal parts and save them in ascending order

        for i in range(3):
            for j in range(3):
                start_x = j * crop_width
                start_y = i * crop_height
                end_x = start_x + crop_width
                end_y = start_y + crop_height

                cropped_image = original_image[start_y:end_y, start_x:end_x]

                # Save the cropped image
                save_path = os.path.join('/content/dataset/1080p/', f'{counter:03d}.jpg')
                cv2.imwrite(save_path, cropped_image)

                counter += 1


# Print the size of the cropped image
cropped_height, cropped_width, _ = cropped_image.shape
print(f"Cropped Image {counter} ({filename}) size: {cropped_width}x{cropped_height}")
print(counter)


Cropped Image 1423 (032.jpg) size: 640x360
1423


In [12]:
#360p
import os
import cv2

# Specify the directory containing the original scaled 360p to 1080p images
original_dir = '/content/scaled/'

# Create a directory to save the cropped images
#os.makedirs('path/to/save/cropped/images', exist_ok=True)
counter = 1
# Iterate over the images in the original directory
for filename in os.listdir(original_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions if needed
        # Load the original 1080p image
        original_path = os.path.join(original_dir, filename)
        original_image = cv2.imread(original_path)

        # Calculate the dimensions of each cropped part
        image_height, image_width, _ = original_image.shape
        crop_height = image_height // 3
        crop_width = image_width // 3

        # Crop the image into 9 equal parts and save them in ascending order
        for i in range(3):
            for j in range(3):
                start_x = j * crop_width
                start_y = i * crop_height
                end_x = start_x + crop_width
                end_y = start_y + crop_height

                cropped_image = original_image[start_y:end_y, start_x:end_x]

                # Save the cropped image
                save_path = os.path.join('/content/dataset/360s/', f'{counter:03d}.jpg')
                cv2.imwrite(save_path, cropped_image)

                counter += 1

# Print the size of the cropped image
cropped_height, cropped_width, _ = cropped_image.shape
print(f"Cropped Image {counter} ({filename}) size: {cropped_width}x{cropped_height}")
print(counter)


Cropped Image 1423 (032.jpg) size: 640x360
1423


model training

In [ ]:
import gc
import os
import numpy as np
import cv2
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate


# Step 1: Organize data and directories
original_dir = '/content/dataset/1080p/'  # Directory containing original 1080p images
scaled_dir = '/content/dataset/360s/'  # Directory containing scaled images

# Step 2: Load and preprocess the images
original_images = []
scaled_images = []

for filename in os.listdir(original_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions if needed
        original_path = os.path.join(original_dir, filename)
        scaled_path = os.path.join(scaled_dir, filename)

        original_image = cv2.imread(original_path)
        scaled_image = cv2.imread(scaled_path)


       # Preprocess the images (resize and normalize)
        original_image = original_image / 255.0  # Resize to 1080p
        scaled_image = scaled_image / 255.0  # Resize to 1080p

        original_images.append(original_image)
        scaled_images.append(scaled_image)

original_images = np.array(original_images)
scaled_images = np.array(scaled_images)

# # Step 3: Define the U-Net model architecture
inputs = Input(shape=(360, 640, 3))  # Update input shape
conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
up5 = UpSampling2D(size=(2, 2))(conv4)

up5 = Conv2D(256, 2, activation='relu', padding='same')(up5)
merge5 = concatenate([conv3, up5], axis=3)
conv5 = Conv2D(256, 3, activation='relu', padding='same')(merge5)
conv5 = Conv2D(256, 3, activation='relu', padding='same')(conv5)
up6 = UpSampling2D(size=(2, 2))(conv5)

up6 = Conv2D(128, 2, activation='relu', padding='same')(up6)
merge6 = concatenate([conv2, up6], axis=3)
conv6 = Conv2D(128, 3, activation='relu', padding='same')(merge6)
conv6 = Conv2D(128, 3, activation='relu', padding='same')(conv6)
up7 = UpSampling2D(size=(2, 2))(conv6)

up7 = Conv2D(64, 2, activation='relu', padding='same')(up7)
merge7 = concatenate([conv1, up7], axis=3)
conv7 = Conv2D(64, 3, activation='relu', padding='same')(merge7)
conv7 = Conv2D(64, 3, activation='relu', padding='same')(conv7)

output = Conv2D(3, 1, activation='sigmoid')(conv7)

model = Model(inputs=inputs, outputs=output)

# Step 4: Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(scaled_images, original_images, batch_size=2, epochs=1, validation_split=0.2)

# Step 5: Save the trained model
model.save('/content/model.h5')
print("model done.")

# Perform memory cleanup
gc.collect()


 10/180 [>.............................] - ETA: 2:04:18 - loss: 0.0862

2nd type model training

In [ ]:
import os
import numpy as np
import cv2
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Step 1: Organize data and directories
original_dir = '/content/dataset/1080p/'  # Directory containing original 1080p images
scaled_dir = '/content/dataset/360s/'  # Directory containing scaled images

# Step 2: Load and preprocess the images
original_images = []
scaled_images = []

for filename in os.listdir(original_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions if needed
        original_path = os.path.join(original_dir, filename)
        scaled_path = os.path.join(scaled_dir, filename)

        original_image = cv2.imread(original_path)
        scaled_image = cv2.imread(scaled_path)

        # Preprocess the images (resize and normalize)
        original_image = original_image / 255.0  # Resize to match scaled image dimensions
        scaled_image = scaled_image / 255.0  # Resize to 1080p

        original_images.append(original_image)
        scaled_images.append(scaled_image)

original_images = np.array(original_images)
scaled_images = np.array(scaled_images)

# Step 3: Define the model architecture
# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(360, 640, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(3, activation='sigmoid'))

# Step 4: Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(scaled_images, original_images, batch_size=2, epochs=5, validation_split=0.2)

# Step 5: Save the trained model
model.save('/content/model.h5')

# Step 6: Evaluate the model (optional)
# test_images = [...]  # Load test images
# predicted_images = model.predict(test_images)

# Step 7: Inference on new scaled images (optional)
# new_scaled_image = cv2.imread('path/to/new/scaled/image.jpg')
# new_scaled_image = cv2.resize(new_scaled_image, (256, 256)) / 255.0
# predicted_image = model.predict(np.array([new_scaled_image]))
# predicted_image = predicted_image[0] * 255.0
# cv2.imwrite('path/to/save/predicted/image.jpg', predicted_image)


In [ ]:
import gc

# Your program code here

# Perform memory cleanup
gc.collect()


0

In [1]:
import torch


if torch.cuda.is_available():
    print('CUDA is available.')
else:
    print('CUDA is not available.')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)


CUDA is available.
Device: cuda


3rd type training with tensor

In [ ]:
import gc
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, ToPILImage, RandomRotation, RandomHorizontalFlip, RandomVerticalFlip
from torchvision.transforms import Normalize
from torchvision.models import resnet50

import time


start_time = time.time()
# Step 1: Organize data and directories
original_dir = '/content/dataset/1080p/'  # Directory containing original 1080p images
scaled_dir = '/content/dataset/360s/'  # Directory containing scaled images

# Step 2: Load and preprocess the images
original_images = []
scaled_images = []

for filename in os.listdir(original_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions if needed
        original_path = os.path.join(original_dir, filename)
        scaled_path = os.path.join(scaled_dir, filename)

        original_image = cv2.imread(original_path)
        scaled_image = cv2.imread(scaled_path)

        # Preprocess the images (normalize)
        original_image = original_image.astype(np.float32) / 255.0
        scaled_image = scaled_image.astype(np.float32) / 255.0

        original_images.append(original_image)
        scaled_images.append(scaled_image)

original_images = np.array(original_images)
scaled_images = np.array(scaled_images)

original_images = original_images.transpose((0, 3, 1, 2))  # Transpose from (num_samples, 640, 360, 3) to (num_samples, 3, 640, 360)
scaled_images = scaled_images.transpose((0, 3, 1, 2))  # Transpose from (num_samples, 640, 360, 3) to (num_samples, 3, 640, 360)

# Move the model to GPU if available
device = torch.device('cuda') #torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert the input data to the correct shape and move to GPU
original_images = torch.from_numpy(original_images).to(device).float()
scaled_images = torch.from_numpy(scaled_images).to(device).float()

# Define the U-Net model architecture
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        # Encoder
        self.encoder1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.encoder2 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.encoder3 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.encoder4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

        # Decoder
        self.decoder4 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )
        self.decoder3 = nn.Sequential(
            nn.Conv2d(512, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )
        self.decoder2 = nn.Sequential(
            nn.Conv2d(256, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )
        self.decoder1 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3, kernel_size=1)
        )

    def forward(self, x):
        # Encoder
        x1 = self.encoder1(x)
        x2 = self.encoder2(x1)
        x3 = self.encoder3(x2)
        x4 = self.encoder4(x3)

        # Decoder
        d4 = self.decoder4(x4)
        d3 = self.decoder3(torch.cat([d4, x3], dim=1))
        d2 = self.decoder2(torch.cat([d3, x2], dim=1))
        d1 = self.decoder1(torch.cat([d2, x1], dim=1))

        return d1


# Create an instance of the U-Net model
model = UNet()

# Move the model to GPU if available
model.to(device)

# Print the model architecture
print(model)


# Define the loss function
criterion = nn.MSELoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the data augmentation and normalization transformations
transform = nn.Sequential(
    RandomRotation(degrees=15),
    RandomHorizontalFlip()
)

# Define the dataset class
class ImageDataset(Dataset):
    def __init__(self, original_images, scaled_images, transform=None):
        self.original_images = original_images
        self.scaled_images = scaled_images
        self.transform = transform

    def __len__(self):
        return len(self.original_images)

    def __getitem__(self, idx):
        original_image = self.original_images[idx]
        scaled_image = self.scaled_images[idx]

        if self.transform:
            original_image = self.transform(original_image)
            scaled_image = self.transform(scaled_image)

        return original_image, scaled_image

# Create the dataset object
dataset = ImageDataset(original_images, scaled_images, transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create the data loaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)

# Train the model
num_epochs = 15

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode

    running_loss = 0.0

    for original_images, scaled_images in train_loader:
        optimizer.zero_grad()  # Clear the gradients

        # Forward pass
        outputs = model(original_images)
        loss = criterion(outputs, scaled_images)

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Compute the average loss for the epoch
    train_loss = running_loss / len(train_loader)

    # Evaluate the model on the validation set
    model.eval()  # Set the model to evaluation mode

    val_loss = 0.0

    with torch.no_grad():
        for original_images, scaled_images in val_loader:
            outputs = model(original_images)
            loss = criterion(outputs, scaled_images)

            val_loss += loss.item()

    val_loss /= len(val_loader)

    # Print the epoch statistics
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'model.pth')

print("model saved.")

# Clear GPU memory
del original_images, scaled_images, model, criterion, optimizer
gc.collect()
torch.cuda.empty_cache()
# End the timer
end_time = time.time()

# Calculate the execution time
execution_time = end_time - start_time

print(f"Execution time: {execution_time/60} minutes.")



3.o prediction

In [3]:
import numpy as np
from PIL import Image

# Load the trained model
model = UNet()
model.load_state_dict(torch.load('/content/model.pth'))

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Load and preprocess the input image
input_image = cv2.imread('/content/dataset/360s/045.jpg')
input_image = cv2.resize(input_image, (640, 360))
input_tensor = ToTensor()(input_image).unsqueeze(0).to(device)

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation
with torch.no_grad():
    # Forward pass
    output_tensor = model(input_tensor)

# Convert the output tensor to a numpy array
output_image = output_tensor.squeeze(0).cpu().numpy()

# Convert the output tensor to a numpy array
output_image = output_tensor.squeeze(0).cpu().numpy()

# Transpose the output array to match the shape (height, width, channels)
output_image = np.transpose(output_image, (1, 2, 0))

# Scale the pixel values from [0, 1] to [0, 255]
output_image = (output_image * 255).astype(np.uint8)

# Convert the output array to a PIL image
output_image = Image.fromarray(output_image)

# Save the predicted image
output_image.save('/content/predicted/predicted_image.jpg')


3.o comparing

In [4]:
import numpy as np
from PIL import Image

# Load the original and predicted images
original_image = Image.open('/content/dataset/1080p/045.jpg')
predicted_image = Image.open('/content/predicted/predicted_image.jpg')

# Convert images to numpy arrays
original_array = np.array(original_image)
predicted_array = np.array(predicted_image)

# Calculate the pixel-wise difference
diff_array = np.abs(original_array - predicted_array)

# Calculate the number of mismatched pixels
num_mismatched_pixels = np.sum(diff_array > 0)

# Calculate the total number of pixels
total_pixels = original_array.size

# Calculate the accuracy as the percentage of matching pixels
accuracy = ((total_pixels - num_mismatched_pixels) / total_pixels) * 100

print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 0.16%


3.1 model training

In [ ]:
import gc
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from torchvision.transforms import ToPILImage


# Step 1: Organize data and directories
original_dir = '/content/dataset/1080p/'  # Directory containing original 1080p images
scaled_dir = '/content/dataset/360s/'  # Directory containing scaled images

# Step 2: Load and preprocess the images
original_images = []
scaled_images = []

for filename in os.listdir(original_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions if needed
        original_path = os.path.join(original_dir, filename)
        scaled_path = os.path.join(scaled_dir, filename)

        original_image = cv2.imread(original_path)
        scaled_image = cv2.imread(scaled_path)

        # Preprocess the images (resize and normalize)
        original_image = cv2.resize(original_image, (640, 360))
        scaled_image = cv2.resize(scaled_image, (640, 360))

        original_images.append(original_image)
        scaled_images.append(scaled_image)

original_images = np.array(original_images)
scaled_images = np.array(scaled_images)

original_images = original_images.transpose((0, 3, 1, 2))  # Transpose from (num_samples, 640, 360, 3) to (num_samples, 3, 640, 360)
scaled_images = scaled_images.transpose((0, 3, 1, 2))  # Transpose from (num_samples, 640, 360, 3) to (num_samples, 3, 640, 360)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert the input data to the correct shape and move to GPU
original_images = torch.from_numpy(original_images).to(device).float()
scaled_images = torch.from_numpy(scaled_images).to(device).float()

# Define the U-Net model architecture
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        # Encoder
        self.encoder1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.encoder2 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.encoder3 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.encoder4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

        # Decoder
        self.decoder4 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )
        self.decoder3 = nn.Sequential(
            nn.Conv2d(512, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )
        self.decoder2 = nn.Sequential(
            nn.Conv2d(256, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )
        self.decoder1 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3, kernel_size=1)
        )

    def forward(self, x):
        # Encoder
        x1 = self.encoder1(x)
        x2 = self.encoder2(x1)
        x3 = self.encoder3(x2)
        x4 = self.encoder4(x3)

        # Decoder
        d4 = self.decoder4(x4)
        d3 = self.decoder3(torch.cat([d4, x3], dim=1))
        d2 = self.decoder2(torch.cat([d3, x2], dim=1))
        d1 = self.decoder1(torch.cat([d2, x1], dim=1))

        return d1


# Create an instance of the U-Net model
model = UNet()

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Print the model architecture
print(model)

# Step 5: Compile and train the model
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, inputs, targets, transform=None):
        self.inputs = inputs
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_image = self.inputs[idx]
        target_image = self.targets[idx]

        # Convert tensors to PIL images
        input_image = ToPILImage()(input_image)
        target_image = ToPILImage()(target_image)

        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)

        return input_image, target_image


# Create dataloaders for training and validation
train_dataset = CustomDataset(scaled_images, original_images, transform=ToTensor())
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Training loop
for epoch in range(1):
    running_loss = 0.0
    for inputs, targets in train_dataloader:
        inputs = inputs.to(device).float()  # Convert input data to float
        targets = targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch: {epoch+1}, Loss: {running_loss}")

# Step 6: Save the trained model
torch.save(model.state_dict(), '/content/model.pth')
print("Model saved.")

# Perform memory cleanup
gc.collect()

3.1 prediction

In [ ]:
import numpy as np
from PIL import Image

# Load the trained model
model = UNet()
model.load_state_dict(torch.load('/content/model.pth'))

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Load and preprocess the input image
input_image = cv2.imread('/content/dataset/360s/045.jpg')
input_image = cv2.resize(input_image, (640, 360))
input_tensor = ToTensor()(input_image).unsqueeze(0).to(device)

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation
with torch.no_grad():
    # Forward pass
    output_tensor = model(input_tensor)

# Convert the output tensor to a numpy array
output_image = output_tensor.squeeze(0).cpu().numpy()

# Convert the output tensor to a numpy array
output_image = output_tensor.squeeze(0).cpu().numpy()

# Transpose the output array to match the shape (height, width, channels)
output_image = np.transpose(output_image, (1, 2, 0))

# Scale the pixel values from [0, 1] to [0, 255]
output_image = (output_image * 255).astype(np.uint8)

# Convert the output array to a PIL image
output_image = Image.fromarray(output_image)

# Save the predicted image
output_image.save('/content/predicted/predicted_image.jpg')


3.1 comparision

In [ ]:
import numpy as np
from PIL import Image

# Load the original and predicted images
original_image = Image.open('/content/dataset/1080p/045.jpg')
predicted_image = Image.open('/content/predicted/predicted_image.jpg')

# Convert images to numpy arrays
original_array = np.array(original_image)
predicted_array = np.array(predicted_image)

# Calculate the pixel-wise difference
diff_array = np.abs(original_array - predicted_array)

# Calculate the number of mismatched pixels
num_mismatched_pixels = np.sum(diff_array > 0)

# Calculate the total number of pixels
total_pixels = original_array.size

# Calculate the accuracy as the percentage of matching pixels
accuracy = ((total_pixels - num_mismatched_pixels) / total_pixels) * 100

print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 0.16%


saving the model to drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

file_path = '/content/my_file.txt'  # Specify the path to your file
drive_path = '/content/drive/MyDrive/my_folder/my_file.txt'  # Specify the destination path in Google Drive

# Copy the file to Google Drive
!cp $file_path $drive_path


predicting

In [ ]:
# Step 6: Evaluate the model (optional)
#test_images = "/content/scaled/022.jpg"  # Load test images
#predicted_images = model.predict(test_images)

# Step 7: Inference on new scaled images (optional)
new_scaled_image = cv2.imread('/content/scaled/022.jpg')
new_scaled_image = cv2.resize(new_scaled_image, (1920, 1080)) / 255.0  # Resize to 1080p and normalize
predicted_image = model.predict(np.array([new_scaled_image]))
predicted_image = predicted_image[0] * 255.0
cv2.imwrite('/content/image.jpg', predicted_image)

print("done.")

Evaluating

In [ ]:
import os
import numpy as np
import cv2
import tensorflow
from tensorflow.keras.models import load_model
from skimage.metrics import mean_squared_error, structural_similarity

def compare_images(original_dir, predicted_dir):
    mse_total = 0
    ssim_total = 0
    num_images = 0

    for filename in os.listdir(original_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions if needed
            original_path = os.path.join(original_dir, filename)
            predicted_path = os.path.join(predicted_dir, filename)

            original_image = cv2.imread(original_path)
            predicted_image = cv2.imread(predicted_path)

            original_image = cv2.resize(original_image, (256, 256))
            predicted_image = cv2.resize(predicted_image, (256, 256))

            mse = mean_squared_error(original_image, predicted_image)
            ssim = structural_similarity(original_image, predicted_image, multichannel=True)

            mse_total += mse
            ssim_total += ssim
            num_images += 1

    mse_avg = mse_total / num_images
    ssim_avg = ssim_total / num_images

    accuracy = 1.0 - mse_avg  # Inverse MSE is used as accuracy metric

    return accuracy, mse_avg, ssim_avg

# Paths to the original and predicted image directories
original_dir = '/content/Pre_orginal/'
predicted_dir = '/content/predicted/'

# Load the trained model
model = load_model('/content/model.h5')

# Make predictions on new images and save them to the predicted directory
# new_images = [...]  # Load new images for prediction
# for i, image in enumerate(new_images):
#     predicted_image = model.predict(np.array([image]))
#     predicted_image = np.squeeze(predicted_image) * 255.0
#     cv2.imwrite(os.path.join(predicted_dir, f'predicted_{i}.jpg'), predicted_image)

# Compare the predicted images with the original images and calculate accuracy
accuracy, mse, ssim = compare_images(original_dir, predicted_dir)
print(f"Accuracy: {accuracy:.4f}")
print(f"MSE: {mse:.4f}")
print(f"SSIM: {ssim:.4f}")


checkin that cropped image and coresponding region of cropped image in original ti know is there any pixel or quality loss

In [ ]:
!wget -O nivetha.jpg "https://pbs.twimg.com/media/FgiNhi1aAAAm43h?format=jpg&name=large"

In [ ]:
import cv2
import numpy as np

# Load the original image
original_image = cv2.imread('/content/nivetha.jpg')

# Define the region of interest (ROI) coordinates
x = 100  # starting x-coordinate
y = 200  # starting y-coordinate
width = 300  # width of the ROI
height = 200  # height of the ROI

# Crop the image based on the ROI
cropped_image = original_image[y:y+height, x:x+width]

# Compare pixel values of the cropped image with the corresponding region in the original image
are_same = np.array_equal(cropped_image, original_image[y:y+height, x:x+width])

# Print the result
if are_same:
    print("The pixel values of the cropped image and the corresponding region in the original image are the same.")
else:
    print("The pixel values of the cropped image and the corresponding region in the original image are different.")
